<div align="center">

# Proyecto 3

## Sede central de Cartago

## Escuela de Ingeniería en Computación

## Inteligencia artificial (Gr 02 – IC6200)

### Regresión lineal con descenso del gradiente y selección de características

### Creado por

Esteban Pérez Picado – 2021046572

Joselyn Montero Rodríguez - 2022136356

Manuel A. Rodríguez Murillo - 2021028686

### 24-06-2025

### Semestre I

### Steven Andrey Pacheco Portuguez
</div>

## Introducción
...

## Importación de librerías

In [1]:
# Python
import os
import cv2

# Visualización y datos
import numpy as np
import pandas as pd

# PyTorch
import torch

# Utilidades
from tqdm import tqdm
from joblib import Parallel, delayed


In [2]:
# Conexion con drive
driveBit = 0
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = "/content/drive/MyDrive/archive"
    driveBit = 1
except ImportError:
    #Jupyter local
    #current_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
    #base_path = os.path.join(current_dir, "data")
    print("Colab no detectado, usando ruta local:", base_path)

Mounted at /content/drive


In [3]:
dataset_path = base_path
csv_path = os.path.join(base_path, "BUTTERFLIES.csv")

In [4]:
# Se verifica que se esté trabajando con la gpu
device = torch.device("cuda" if torch.cuda.is_available () else "cpu")
print ("Is Torch Cuda Available? " + str (torch.cuda.is_available()))

Is Torch Cuda Available? True


## Selección de especies para el clasificador

In [5]:
# Cargar el archivo CSV que contiene las etiquetas de las imágenes
data = pd.read_csv(csv_path)

# Contar el número de muestras por especie
species_counts = data["labels"].value_counts()

# Obtener las 30 especies con mayor cantidad de muestras
top_30_species = species_counts.head(30).index.tolist()

print("Lista de las 30 especies con mayor cantidad de muestras:")
print(top_30_species)



Lista de las 30 especies con mayor cantidad de muestras:
['MOURNING CLOAK', 'GREEN HAIRSTREAK', 'BROWN ARGUS', 'BROOKES BIRDWING', 'SLEEPY ORANGE', 'CHALK HILL BLUE', 'HUMMING BIRD HAWK MOTH', 'WHITE LINED SPHINX MOTH', 'ATALA', 'BROWN SIPROETA', 'ARCIGERA FLOWER MOTH', 'BANDED ORANGE HELICONIAN', 'SCARCE SWALLOW', 'CRECENT', 'HERCULES MOTH', 'MILBERTS TORTOISESHELL', 'ROSY MAPLE MOTH', 'RED CRACKER', 'ORANGE TIP', 'BANDED TIGER MOTH', 'Iphiclus sister', 'CHECQUERED SKIPPER', 'EASTERN PINE ELFIN', 'EMPEROR GUM MOTH', 'DANAID EGGFLY', 'GREAT JAY', 'COPPER TAIL', 'EASTERN COMA', 'CLEOPATRA', 'BIRD CHERRY ERMINE MOTH']


## Reducción de dimensionalidad de imágenes

In [10]:
# Reduce la dimensión de las imágenes a 128x128 píxeles
def resize_image(img_path, size=(128, 128)):
    img = cv2.imread(img_path)
    if img is not None:
        resized_img = cv2.resize(img, size)
        return resized_img
    else:
        print(f"Error al cargar la imagen: {img_path}")
        return None

In [22]:
def process_images(base_path, top_species):
    """
    Reduce el tamaño de las imágenes de las 30 especies con mayor cantidad de muestras,
    y las almacena en una nueva carpeta llamada "species_selected"
    """
    # Crear la carpeta "species_selected" si no existe
    selected_species_path = os.path.join(base_path, "species_selected")
    os.makedirs(selected_species_path, exist_ok=True)

    # Recorrer las carpetas de train, test y valid
    for dir in ['train', 'test', 'valid']:
        dataset_path = os.path.join(base_path, dir)

        # Crear la carpeta para el tipo de dataset
        processed_path = os.path.join(selected_species_path, dir)
        os.makedirs(processed_path, exist_ok=True)

        # Verificar si la carpeta ya fue procesada
        if os.listdir(processed_path):  # Si hay archivos en la carpeta, ya fue procesada
            print(f"Las imágenes en '{dir}' ya han sido procesadas.")
            continue  # Saltar a la siguiente carpeta si ya fue procesada

        print(f"Procesando imágenes en '{dir}'...")

        # Recorrer las carpetas de especies
        for species in os.listdir(dataset_path):
            if species in top_species:
                species_path = os.path.join(dataset_path, species)

                # Crear una carpeta para la especie en "species_selected"
                species_selected_path = os.path.join(processed_path, species)
                os.makedirs(species_selected_path, exist_ok=True)

                # Recorrer las imágenes de la especie
                for img_filename in os.listdir(species_path):
                    img_path = os.path.join(species_path, img_filename)

                    # Redimensionar la imagen
                    resized_img = resize_image(img_path)
                    if resized_img is not None:

                        # Guardar la imagen redimensionada
                        output_path = os.path.join(species_selected_path, img_filename)
                        cv2.imwrite(output_path, resized_img)

        print(f"Finalizado el procesamiento de imágenes en '{dir}'.")

process_images(dataset_path, top_30_species)

Las imágenes en 'train' ya han sido procesadas.
Las imágenes en 'test' ya han sido procesadas.
Las imágenes en 'valid' ya han sido procesadas.


In [18]:
species_selected_path = os.path.join(dataset_path, "species_selected")

## Incorporación de muestras al set de testing